# Análisis Descriptivo ENO

### 1. Introducción

Este notebook presenta un análisis descriptivo y visual de los datos sobre las enfermedades de notificación obligatoria (ENO) en Chile. Exploraremos la distribución de las enfermedades, su evolución temporal, y otros factores relevantes.

Descripción del conjunto de datos: El Departamento de Epidemiología del Ministerio de Salud de Chile pone a disposición pública, archivo con bases de datos de algunas Enfermedades de Notificación Obligatoria (ENO) que se encuentran bajo vigilancia epidemiológica según lo establece el Decreto Supremo N°7/2019 MINSAL. Los datos corresponden al periodo 2007- 2022. Las ENO en la base de datos son: Arbovirus, Botulismo Adulto, Botulismo Infantil, Brucelosis, Carbunco, Chagas, Cisticercosis, Coqueluche, Creutzfeldt-Jakob, Difteria, Enfermedad meningocócica, Fiebre Tifoidea y Paratifoidea, Fiebre Q, Hepatitis A, Hepatitis B, Hepatitis C, Hepatitis E, Hidatidosis (Equinococosis), Infección gonocócica, Lepra, Leptospirosis, Malaria, Meningitis Bacteriana, MPOX, Parotiditis, Psitacosis, Sarampión, Sífilis, Síndrome Pulmonar por Hantavirus, Tétanos, Triquinosis, VIH.

Consideraciones:

- Los años 2019 al 2022 están sujetos a validación, por lo que podrían ser modificados en la siguiente actualización de datos. 
- Existen algunos campos que se encuentran vacíos desde el 2007-2018 debido a cambios realizados en los sistemas de información de notificación.

### 2. Carga y Exploración de Datos

In [36]:
# Librerías
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from scipy import stats

# inhabilita warnings
import warnings
warnings.filterwarnings("ignore")

In [37]:
df = pd.read_csv('C:/Users/aalei/Desktop/github_dq/Datos/ENO Notification/20240304_base_eno_final.csv', delimiter=';')
print('Rows: ', df.shape[0], '  Columns: ', df.shape[1])
df.head(2)

Rows:  294352   Columns:  43


,etapa_clinica,region,seremi,nacionalidad,cie_10_diagnostico,diagnostico,Autóctono,anho_notificacion,ENO,pueblo_indigena,...,genotipo_1,recuento_linfocitos,clasificacion_final_vih,etapa_vih_sida,hospitalizacion,nombre_primer_sintoma,SeroGrupo,sexo,grupo_edad,codigo_comuna_residencia
0,CONFIRMADA,Región de Tarapacá,TARAPACÁ,Extranjero,A97.0,Dengue sin datos de alarma,No,2020,Arbovirus,Desconocido,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mujer,25-29,01107
1,CONFIRMADA,Región de Valparaíso,VALPARAÍSO,Desconocido,A90.X,Fiebre del dengue,No,2018,Arbovirus,Desconocido,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hombre,70-74,05301


In [38]:
df.tail(2)

,etapa_clinica,region,seremi,nacionalidad,cie_10_diagnostico,diagnostico,Autóctono,anho_notificacion,ENO,pueblo_indigena,...,genotipo_1,recuento_linfocitos,clasificacion_final_vih,etapa_vih_sida,hospitalizacion,nombre_primer_sintoma,SeroGrupo,sexo,grupo_edad,codigo_comuna_residencia
294350,CONFIRMADA,Región Metropolitana de Santiago,RM,Extranjero,Z22.8,PORTADOR DE OTRAS ENFERMEDADES INFECCIOSAS,NaN,2020,Chagas,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mujer,25-29,13101
294351,CONFIRMADA,Región Metropolitana de Santiago,RM,Chile,Z22.8,PORTADOR DE OTRAS ENFERMEDADES INFECCIOSAS,NaN,2022,Chagas,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hombre,15-19,13101


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294352 entries, 0 to 294351
Data columns (total 43 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   etapa_clinica             294352 non-null  object 
 1   region                    294352 non-null  object 
 2   seremi                    294300 non-null  object 
 3   nacionalidad              294352 non-null  object 
 4   cie_10_diagnostico        243970 non-null  object 
 5   diagnostico               243928 non-null  object 
 6   Autóctono                 201 non-null     object 
 7   anho_notificacion         294352 non-null  int64  
 8   ENO                       294352 non-null  object 
 9   pueblo_indigena           294352 non-null  object 
 10  nombre_instruccion        294352 non-null  object 
 11  anio_aprobado             57001 non-null   float64
 12  presenta_sintomas         4132 non-null    object 
 13  sintomas                  239 non-null     o

In [40]:
# Datos int
df.describe().T

,count,mean,std,min,25%,50%,75%,max
anho_notificacion,294352.0,2015.718116,4.321443,2007.0,2012.0,2017.0,2019.0,2022.0
anio_aprobado,57001.0,2.980176,1.981536,1.0,1.0,3.0,4.0,10.0
n_parejas_sexuales,66841.0,3.021005,18.004775,0.0,1.0,1.0,2.0,1000.0
edad,17990.0,23.692385,14.361373,0.0,14.0,22.0,30.0,96.0
recuento_linfocitos,42617.0,344.081165,1837.719131,-29.0,102.0,279.0,461.0,230420.0


In [53]:
#Unique para observar erratas en literales
for i in df:
    print('VARIABLE: ' + i)
    print(df[i].unique())
    print(' --- ')

VARIABLE: etapa_clinica
['CONFIRMADA' 'PROBABLE']
 --- 
VARIABLE: region
['Región de Tarapacá' 'Región de Valparaíso'
 'Región Metropolitana de Santiago' 'Región de Los Lagos'
 'Región de Arica y Parinacota' 'Región del Biobío' 'Región de Los Ríos'
 'Región de Antofagasta' 'Región de la Araucanía'
 'Región del Libertador General Bernardo OHiggins'
 'Región de Aysén del General Carlos Ibáñez del Campo' 'Región del Ñuble'
 'Región del Maule' 'Región de Atacama' 'Región de Coquimbo'
 'Región de Magallanes y la Antártica Chilena']
 --- 
VARIABLE: seremi
['TARAPACÁ' 'VALPARAÍSO' 'RM' 'LOS LAGOS' 'ARICA Y PARINACOTA' 'BIOBÍO'
 'LOS RÍOS' 'ANTOFAGASTA' 'ARAUCANÍA' 'OHIGGINS' 'AYSÉN' 'ÑUBLE' 'MAULE'
 'ATACAMA' 'COQUIMBO' 'MAGALLANES' nan]
 --- 
VARIABLE: nacionalidad
['Extranjero' 'Desconocido' 'Chile']
 --- 
VARIABLE: cie_10_diagnostico
['A97.0' 'A90.X' 'A97.9' 'A90' 'A97.2' 'A97.1' 'U06.9' 'A92.0' 'A95.9'
 'A23.1' 'A23.9' 'A23.3' 'A23.8' 'A23.0' 'A23' 'B69.0' 'A37.0' 'A37.1'
 'A37.9' 'A37' '

### 3. Preprocessing

df['codigo_comuna_residencia'].unique()

In [42]:
df['codigo_comuna_residencia'].unique()

array(['01107', '05301', '05109', '13114', '13104', '13129', '05107',
       '13123', '15101', '13201', '12302', '13125', '16101', '14101',
       '13113', '10109', '10209', '13301', '13101', '08101', '13403',
       '13128', '10101', '13120', '05101', '13115', '08107', '01101',
       '08301', '13108', '02101', '09101', '10301', '02104', '05701',
       '13127', '15***', '02***', '05***', '05201', '13132', '13105',
       '13302', '05804', '13***', '13117', '13110', '06113', '10305',
       '07101', '13122', '05706', '111**', '10107', '13119', '09208',
       '13118', '13111', '10104', '07401', '08106', '13130', '13106',
       '06108', '07403', '13303', '06302', '10201', '08108', '09119',
       '*****', '05802', '13402', '14104', '13103', '09110', '13107',
       '10307', '07301', '06101', '11101', '07203', '05103', '16106',
       '03102', '16302', '07302', '07102', '03301', '103**', '10205',
       '09201', '04102', '073**', '161**', '082**', '03103', '162**',
       '10202', '083

In [43]:
# Función para obtener la moda, ignorando los valores 'Desconocido'

def moda_sin_desconocido(grupo):
    return grupo[grupo != 'Desconocido'].mode()[0] if not grupo[grupo != 'Desconocido'].mode().empty else 'Desconocido'

# Calcular la moda de 'region' agrupada por 'codigo_comuna_residencia'
modas_por_comuna = df.groupby('codigo_comuna_residencia')['region'].transform(moda_sin_desconocido)

# Reemplazar 'Desconocido' por la moda correspondiente
df['region'] = df.apply(lambda row: modas_por_comuna[row.name] if row['region'] == 'Desconocido' else row['region'], axis=1)


In [44]:
df[df['seremi'].isnull()]

,etapa_clinica,region,seremi,nacionalidad,cie_10_diagnostico,diagnostico,Autóctono,anho_notificacion,ENO,pueblo_indigena,...,genotipo_1,recuento_linfocitos,clasificacion_final_vih,etapa_vih_sida,hospitalizacion,nombre_primer_sintoma,SeroGrupo,sexo,grupo_edad,codigo_comuna_residencia
128356,CONFIRMADA,Región de Antofagasta,NaN,Extranjero,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2009,Sarampion,No pertenece,...,INDETERMINADO,NaN,NaN,NaN,NaN,NaN,NaN,hombre,05-09,02101
128357,CONFIRMADA,Región Metropolitana de Santiago,NaN,Chile,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2011,Sarampion,Desconocido,...,D4,NaN,NaN,NaN,NaN,NaN,NaN,mujer,00-04,13401
128358,CONFIRMADA,Región Metropolitana de Santiago,NaN,Chile,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2011,Sarampion,Desconocido,...,D4,NaN,NaN,NaN,NaN,NaN,NaN,mujer,35-39,13401
128359,CONFIRMADA,Región Metropolitana de Santiago,NaN,Chile,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2011,Sarampion,Desconocido,...,D4,NaN,NaN,NaN,NaN,NaN,NaN,hombre,30-34,13401
128360,CONFIRMADA,Región Metropolitana de Santiago,NaN,Chile,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2011,Sarampion,No pertenece,...,D4,NaN,NaN,NaN,NaN,NaN,NaN,mujer,45-49,13114
128361,CONFIRMADA,Región Metropolitana de Santiago,NaN,Chile,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2011,Sarampion,No pertenece,...,D4,NaN,NaN,NaN,NaN,NaN,NaN,mujer,35-39,13114
128362,CONFIRMADA,Región Metropolitana de Santiago,NaN,Chile,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2011,Sarampion,No pertenece,...,D9,NaN,NaN,NaN,NaN,NaN,NaN,mujer,30-34,13123
128363,CONFIRMADA,Región Metropolitana de Santiago,NaN,Desconocido,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2015,Sarampion,Desconocido,...,H1,NaN,NaN,NaN,NaN,NaN,NaN,hombre,30-34,13101
128364,CONFIRMADA,Región Metropolitana de Santiago,NaN,Chile,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2015,Sarampion,No pertenece,...,H1,NaN,NaN,NaN,NaN,NaN,NaN,hombre,00-04,13501
128365,CONFIRMADA,Región Metropolitana de Santiago,NaN,Extranjero,B05.9,SARAMPION SIN COMPLICACIONES,NaN,2015,Sarampion,No pertenece,...,H1,NaN,NaN,NaN,NaN,NaN,NaN,hombre,35-39,13101


In [45]:
# Número de casos por enfermedad
#df['diagnostico'].value_counts().plot(kind='bar', title='Casos por Enfermedad')


In [46]:
df.isnull().mean() * 100

etapa_clinica                0.000000
region                       0.000000
seremi                       0.017666
nacionalidad                 0.000000
cie_10_diagnostico          17.116242
diagnostico                 17.130510
Autóctono                   99.931714
anho_notificacion            0.000000
ENO                          0.000000
pueblo_indigena              0.000000
nombre_instruccion           0.000000
anio_aprobado               80.635090
presenta_sintomas           98.596239
sintomas                    99.918805
consumo_agua_no_potable     99.990488
antecedente_teniasis        99.990488
imagenologia                99.990488
manifestaciones             98.593521
existe_coinfeccion          64.792833
via_transmision             69.165489
n_parejas_sexuales          77.292154
uso_preservativo            74.476137
pais_contagio                0.000000
edad                        93.888270
puerpera                    99.961611
paciente_inmunosuprimido    99.530494
vih         